In [1]:
import pandas as pd
import numpy as np
from datetime import *


In [13]:
"""
hierarchy is created in here.
Hierarchy is indicated by additional expressions next to the string,
such as Camper-FW -> Camper-FW-10 or Camper-FW-10 -> Camper-FW-10-1' .

"""


hier3 = month_season_sales_channel_country_cluster['target'].unique().tolist()
hier2 = month_season_sales_channel['target'].unique().tolist() 
hier1 = month_season_company['target'].unique().tolist()

final_list = hier1 + hier2 + hier3  #Here the unique nodes of each hierarchy level are listed.

df = pd.DataFrame(final_list)
df.rename(columns = {0:'child'}, inplace = True)

df['new name'] = df['child'].apply(lambda x : ','.join(reversed(x.split('-'))))

df['text_new1'] = [x.split(',')[1:] for x in df['new name']]


df['text_new2'] = [','.join(map(str, l)) for l in df['text_new1']]

df['parent'] = df['text_new2'].apply(lambda x : '-'.join(reversed(x.split(','))))
df = df.drop(['new name', 'text_new1', 'text_new2'], axis=1)

df['parent -> child'] = df.apply(
    lambda x: x.child if (x.parent is np.nan) else f'{x.parent}{" -> "}{x.child}', axis=1
    )

hier = df['parent -> child'].unique().tolist()
parents = df.loc[~df.parent.isna()].parent.unique().tolist()
children = df.child.unique().tolist()

btm = [v for v in hier if (len(v.split(' -> ')) > 1) and (v.split(' -> ')[1] not in parents)]
hiersorted = df['child'].to_list()

hier

['Camper -> Camper-FW',
 'Camper -> Camper-SS',
 'Camper-FW -> Camper-FW-10',
 'Camper-FW -> Camper-FW-15',
 'Camper-FW -> Camper-FW-20',
 'Camper-FW -> Camper-FW-32',
 'Camper-FW -> Camper-FW-34',
 'Camper-SS -> Camper-SS-10',
 'Camper-SS -> Camper-SS-15',
 'Camper-SS -> Camper-SS-20',
 'Camper-SS -> Camper-SS-32',
 'Camper-SS -> Camper-SS-34',
 'Camper-FW-10 -> Camper-FW-10-1',
 'Camper-FW-10 -> Camper-FW-10-2',
 'Camper-FW-10 -> Camper-FW-10-3',
 'Camper-FW-15 -> Camper-FW-15-1',
 'Camper-FW-15 -> Camper-FW-15-2',
 'Camper-FW-15 -> Camper-FW-15-3',
 'Camper-FW-20 -> Camper-FW-20-1',
 'Camper-FW-20 -> Camper-FW-20-2',
 'Camper-FW-20 -> Camper-FW-20-3',
 'Camper-FW-32 -> Camper-FW-32-1',
 'Camper-FW-32 -> Camper-FW-32-2',
 'Camper-FW-32 -> Camper-FW-32-3',
 'Camper-FW-34 -> Camper-FW-34-1',
 'Camper-FW-34 -> Camper-FW-34-2',
 'Camper-FW-34 -> Camper-FW-34-3',
 'Camper-SS-10 -> Camper-SS-10-1',
 'Camper-SS-10 -> Camper-SS-10-2',
 'Camper-SS-10 -> Camper-SS-10-3',
 'Camper-SS-15 -> Camp

In [14]:
"""
It is the method of summing matrix finding according to the hierarchy order above,
but this algorithm only works for 3 hierarchies, but we have 4 hierarchies, so extra operations will be required.


"""
def middle_sm_vals(col):
    tmp = col.reset_index()
    tmp.loc[tmp['index'].str.contains(col.name.split(' -> ')[0]), col.name] = 1
    return tmp.set_index('index')[col.name]

sm = np.vstack((
            # bottom level
            np.identity(len(btm)),

            # middle hierarchies
            pd.DataFrame(
                index=[h for h in hier if h not in btm and ' -> ' in h], 
                columns=btm, data=0
                ).apply(lambda x: middle_sm_vals(x)).values, 
    
            # top level all 1's
            np.ones(len(btm))))
np.set_printoptions(threshold=np.inf)

summing_mat = np.flip(sm)
summing_mat = np.delete(summing_mat, 11, 0)

store_count=30


In [15]:
"""
The desired summing matrix is obtained by adding and removing some columns manually.

"""

summing_mat = np.delete(summing_mat, 11, 0)
summing_mat.shape

ones = np.ones(15)
zeros = np.zeros(15)
arr1 = np.append(ones,zeros,axis= 0).reshape(1, 30)
arr2 = np.append(zeros,ones,axis= 0).reshape(1, 30)

summing_mat = np.append(summing_mat,arr1,axis= 0)

i, j = 42,1
i, j, s = (i, j, -1) if i<j else (j, i, 1)
summing_mat[i:j+1] = np.roll(summing_mat[i:j+1],shift=s,axis=0)

summing_mat = np.append(summing_mat,arr2,axis= 0)

i, j = 42,2
i, j, s = (i, j, -1) if i<j else (j, i, 1)
summing_mat[i:j+1] = np.roll(summing_mat[i:j+1],shift=s,axis=0)

summing_mat.shape

(43, 30)

In [16]:
"""
REAL and PRED dataframes are created by merging hierarchies. 
These dataframes will be used for reconcilication and comparisons.
"""


datahier1 = merged_datahier1[['date', 'target', 'sales_quantity', 'Pred_quant']]
datahier2 = merged_datahier2[['date', 'target', 'sales_quantity', 'Pred_quant']]
datahier3 = merged_datahier3[['date', 'target', 'sales_quantity', 'Pred_quant']]
vertical_concat = pd.concat([datahier1, datahier2, datahier3], axis=0, ignore_index=True)

VPred = vertical_concat[['date', 'target', 'Pred_quant']]
VReal = vertical_concat[['date', 'target', 'sales_quantity']]

PRED = VPred.pivot(index='target', columns='date', values='Pred_quant')
REAL = VReal.pivot(index='target', columns='date', values='sales_quantity')
totalPRED = PRED.loc[['Camper-FW', 'Camper-SS']].sum()
totalREAL = REAL.loc[['Camper-FW', 'Camper-SS']].sum()
totalPRED.name = "Total"
totalREAL.name = "Total"

PRED = PRED.append(totalPRED, ignore_index=False)
REAL = REAL.append(totalREAL, ignore_index=False)


REAL = REAL.sort_values(by="target", key=lambda x: x.str.len())
PRED = PRED.sort_values(by="target", key=lambda x: x.str.len())
REAL = REAL.sort_values(by="target")
PRED = PRED.sort_values(by="target")

hiersorted.insert(0, 'Total')

REAL = REAL.reindex(hiersorted)
PRED = PRED.reindex(hiersorted)
#REAL = REAL.div(5000)
#PRED = PRED.div(5000)


In [17]:
"""
A different method for the estimation of the covariance matrix.
It is a method of finding a covariance matrix with more mathematical operations.
then beta is obtained from covariance matrix.
"""

np.set_printoptions(suppress=True)
errordf.iloc[:, :30].to_numpy()


n = 51 # number of data series
nn = 30 # number of data series
cov = np.zeros((nn,nn))
xm  = errordf.iloc[:, :51].to_numpy()
print(xm.shape)

# xm
for i in range(nn):
    for j in range(nn):
        x1 = xm[i,:]
        x2 = xm[j,:]
        x1_mean = x1.mean()
        x2_mean = x2.mean()        
        for k in range(n):
            cov[i,j]+= (x1[k]-x1_mean)*(x2[k]-x2_mean)
        cov[i,j]/=n
        
print(cov.shape)
var = np.diag(np.diag(cov))

cov_mat = summing_mat @ var @ summing_mat.T 

inv_cov = np.linalg.pinv(cov_mat)

c = np.linalg.pinv(summing_mat.T @ inv_cov @ summing_mat)

beta = c @ summing_mat.T @ inv_cov
print(beta)


(30, 51)
(30, 30)
[[ 0.02040816  0.03651987 -0.01611171  0.20730397 -0.04269603 -0.04269603
  -0.04269603 -0.04269603 -0.00322234 -0.00322234 -0.00322234 -0.00322234
  -0.00322234  0.73576799 -0.26423201 -0.26423201 -0.01423201 -0.01423201
  -0.01423201 -0.01423201 -0.01423201 -0.01423201 -0.01423201 -0.01423201
  -0.01423201 -0.01423201 -0.01423201 -0.01423201 -0.00107411 -0.00107411
  -0.00107411 -0.00107411 -0.00107411 -0.00107411 -0.00107411 -0.00107411
  -0.00107411 -0.00107411 -0.00107411 -0.00107411 -0.00107411 -0.00107411
  -0.00107411]
 [ 0.02040816  0.03651987 -0.01611171  0.20730397 -0.04269603 -0.04269603
  -0.04269603 -0.04269603 -0.00322234 -0.00322234 -0.00322234 -0.00322234
  -0.00322234 -0.26423201  0.73576799 -0.26423201 -0.01423201 -0.01423201
  -0.01423201 -0.01423201 -0.01423201 -0.01423201 -0.01423201 -0.01423201
  -0.01423201 -0.01423201 -0.01423201 -0.01423201 -0.00107411 -0.00107411
  -0.00107411 -0.00107411 -0.00107411 -0.00107411 -0.00107411 -0.00107411
  -0.

In [18]:
"""
The covariance matrix estimation method in the paper.
Then lists are defined for accuracy metrics.

"""

### PAPERDAKİ İLE COV ESTİMATE
np.set_printoptions(suppress=True) 
error_X = errordf.iloc[:, :50].to_numpy() # having errors

n = np.ones((30, 1)) # defining 1 matrix
x = error_X - n @ n.T @ error_X / 30 ### calculation in the formula 
var_cov = x @ x.T / 30
var = np.diag(np.diag(var_cov)) ### finding diagonal matirx of var_cov



### calculations from paper
cov_mat = summing_mat @ var @ summing_mat.T 
inv_cov = np.linalg.pinv(cov_mat)
c = np.linalg.pinv(summing_mat.T @ inv_cov @ summing_mat)
beta = c @ summing_mat.T @ inv_cov

print(beta) 

Y_base_forecast = PRED.iloc[:, 42:]
Y_real = REAL.iloc[:, 42:]


Y_reconciled = summing_mat @ beta @ Y_base_forecast.to_numpy()

Y_base_forecast = Y_base_forecast.to_numpy()
Y_real = Y_real.to_numpy()



MSE_LST_recon = []
BİAS_LST_recon = []
MAE_LST_recon = []
LOST_MARGİN_PCT_recon = []
INV_COST_PCT_recon = []
TOTAL_COST_PCT_recon = []

MSE_LST_base = []
BİAS_LST_base = []
MAE_LST_base = []
LOST_MARGİN_PCT_base = []
INV_COST_PCT_base = []
TOTAL_COST_PCT_base = []




reconcilations = Y_reconciled.flatten()
expected = Y_real.flatten()
predictions = Y_base_forecast.flatten()

reconcilations = reconcilations.clip(min=0)

result = np.hstack((expected, reconcilations, predictions))




[[ 0.02040816  0.03651987 -0.01611171  0.20730397 -0.04269603 -0.04269603
  -0.04269603 -0.04269603 -0.00322234 -0.00322234 -0.00322234 -0.00322234
  -0.00322234  0.73576799 -0.26423201 -0.26423201 -0.01423201 -0.01423201
  -0.01423201 -0.01423201 -0.01423201 -0.01423201 -0.01423201 -0.01423201
  -0.01423201 -0.01423201 -0.01423201 -0.01423201 -0.00107411 -0.00107411
  -0.00107411 -0.00107411 -0.00107411 -0.00107411 -0.00107411 -0.00107411
  -0.00107411 -0.00107411 -0.00107411 -0.00107411 -0.00107411 -0.00107411
  -0.00107411]
 [ 0.02040816  0.03651987 -0.01611171  0.20730397 -0.04269603 -0.04269603
  -0.04269603 -0.04269603 -0.00322234 -0.00322234 -0.00322234 -0.00322234
  -0.00322234 -0.26423201  0.73576799 -0.26423201 -0.01423201 -0.01423201
  -0.01423201 -0.01423201 -0.01423201 -0.01423201 -0.01423201 -0.01423201
  -0.01423201 -0.01423201 -0.01423201 -0.01423201 -0.00107411 -0.00107411
  -0.00107411 -0.00107411 -0.00107411 -0.00107411 -0.00107411 -0.00107411
  -0.00107411 -0.001074

In [23]:
"""
BİAS calculation
"""


forecast_errors = [expected[i] - predictions[i] for i in range(len(expected))]
bias_org = sum(forecast_errors) * 1.0/sum(expected)
print('Bias_base: %f' % bias_org)

forecast_errors2 = [expected[i] - reconcilations[i] for i in range(len(expected))]
bias_rec = sum(forecast_errors2) * 1.0/sum(expected)
print('Bias_recon: %f' % bias_rec)
bias_rec = round(bias_rec, 3)
bias_org = round(bias_org, 3)

BİAS_LST_recon.append(bias_rec)
BİAS_LST_base.append(bias_org)


Bias_base: 0.021209
Bias_recon: 0.343535


In [24]:
"""
MAE calculation
"""
forecast_errors3 = [expected[i] - predictions[i] for i in range(len(expected))]
mae1 = sum([abs(number) for number in forecast_errors3]) * 1.0/sum(expected)
print('MAE_base: %f' % mae1)

forecast_errors4 = [expected[i] - reconcilations[i] for i in range(len(expected))]
mae2 = sum([abs(number) for number in forecast_errors4]) * 1.0/sum(expected)
print('MAE_recon: %f' % mae2)

mae2 = round(mae2, 3)
mae1 = round(mae1, 3)   
MAE_LST_recon.append(mae2)
MAE_LST_base.append(mae1)

MAE_base: 0.171742
MAE_recon: 0.721942


In [25]:
"""
INV_COST_PCT, LOST_MARGİN_PCT, TOTAL_COST_PCT calculation for reconcilation
"""

###RECONCİLED
cost_of_good_sold = 1
unit_price = 1.3
unit_margin = unit_price - cost_of_good_sold
x = reconcilations * unit_price

x = x+0.3

x = np.around(x)

x = x.clip(min=1)


estimated_sales = np.minimum(x, expected)

leftover_inventory = np.maximum(0, (x - expected - 1))


estimated_sales_margin_dollar = estimated_sales * unit_margin

leftover_inventory_at_cost = leftover_inventory * cost_of_good_sold
observed_sales_dollar = expected * unit_price
observed_sales_margin_dollar = expected * unit_margin

lost_margin_pct = sum(observed_sales_margin_dollar - estimated_sales_margin_dollar)  / sum(observed_sales_margin_dollar)
inventory_cost_pct = ((sum(leftover_inventory_at_cost) / 52) * 0.2) / (sum(observed_sales_dollar) / 2)
total_cost_pct = lost_margin_pct + inventory_cost_pct
print('RECONCİLATİON')
print('lost_margin_pct:', lost_margin_pct)
print('inventory_cost_pct:', inventory_cost_pct)
print('total_cost_pct:', total_cost_pct)
lost_margin_pct = round(lost_margin_pct, 4)
inventory_cost_pct = round(inventory_cost_pct, 5)
total_cost_pct = round(total_cost_pct, 4)  
LOST_MARGİN_PCT_recon.append(lost_margin_pct)
INV_COST_PCT_recon.append(inventory_cost_pct)
TOTAL_COST_PCT_recon.append(total_cost_pct)  

RECONCİLATİON
lost_margin_pct: 0.42806318028246043
inventory_cost_pct: 0.001665485062184954
total_cost_pct: 0.4297286653446454


In [26]:

"""
INV_COST_PCT, LOST_MARGİN_PCT, TOTAL_COST_PCT calculation for base forecast
"""

cost_of_good_sold = 1
unit_price = 1.3
unit_margin = unit_price - cost_of_good_sold
x = predictions * unit_price

x = x+0.3

x = np.around(x)

x = x.clip(min=1)


estimated_sales = np.minimum(x, expected)

leftover_inventory = np.maximum(0, (x - expected - 1))


estimated_sales_margin_dollar = estimated_sales * unit_margin

leftover_inventory_at_cost = leftover_inventory * cost_of_good_sold
observed_sales_dollar = expected * unit_price
observed_sales_margin_dollar = expected * unit_margin

lost_margin_pct = sum(observed_sales_margin_dollar - estimated_sales_margin_dollar)  / sum(observed_sales_margin_dollar)
inventory_cost_pct = ((sum(leftover_inventory_at_cost) / 52) * 0.2) / (sum(observed_sales_dollar) / 2)
total_cost_pct = lost_margin_pct + inventory_cost_pct
print('BASE')
print('lost_margin_pct:', lost_margin_pct)
print('inventory_cost_pct:', inventory_cost_pct)
print('total_cost_pct:', total_cost_pct)
lost_margin_pct = round(lost_margin_pct, 4)
inventory_cost_pct = round(inventory_cost_pct, 5)
total_cost_pct = round(total_cost_pct, 4)  
LOST_MARGİN_PCT_base.append(lost_margin_pct)
INV_COST_PCT_base.append(inventory_cost_pct)
TOTAL_COST_PCT_base.append(total_cost_pct)


BASE
lost_margin_pct: 0.028757903851498356
inventory_cost_pct: 0.0017818844064100958
total_cost_pct: 0.03053978825790845


In [ ]:
"""
You can get an error here so that a new excel file is not created every time we run the whole code. 
After you run the next cell, you can run this cell.

"""


INDEX =  pd.DataFrame(liste).T
INDEX

In [ ]:
liste = ['MSE_BASE', 'MSE_RECON', 'BİAS_BASE', 'BİAS_RECON', 'MAE_BASE', 'MAE_RECON', 'LOST_MARGİN_BASE','LOST_MARGİN_RECON',
        'INV_COST_BASE','INV_COST_RECON','TOTAL_COST_BASE','TOTAL_COST_RECON']
MSE_LST_recon =  pd.DataFrame(MSE_LST_recon).T
BİAS_LST_recon =  pd.DataFrame(BİAS_LST_recon).T
MAE_LST_recon =  pd.DataFrame(MAE_LST_recon).T
LOST_MARGİN_PCT_recon =  pd.DataFrame(LOST_MARGİN_PCT_recon).T
INV_COST_PCT_recon =  pd.DataFrame(INV_COST_PCT_recon).T
TOTAL_COST_PCT_recon =  pd.DataFrame(TOTAL_COST_PCT_recon).T

MSE_LST_base =  pd.DataFrame(MSE_LST_base).T
BİAS_LST_base =  pd.DataFrame(BİAS_LST_base).T
MAE_LST_base =  pd.DataFrame(MAE_LST_base).T
LOST_MARGİN_PCT_base =  pd.DataFrame(LOST_MARGİN_PCT_base).T
INV_COST_PCT_base =  pd.DataFrame(INV_COST_PCT_base).T
TOTAL_COST_PCT_base =  pd.DataFrame(TOTAL_COST_PCT_base).T

INDEX = INDEX.T

In [ ]:


DF = pd.DataFrame()
DF = pd.concat([MSE_LST_base,MSE_LST_recon,BİAS_LST_base,BİAS_LST_recon,MAE_LST_base,MAE_LST_recon,LOST_MARGİN_PCT_base,
               LOST_MARGİN_PCT_recon,INV_COST_PCT_base,INV_COST_PCT_recon,TOTAL_COST_PCT_base,TOTAL_COST_PCT_recon], ignore_index=False, sort=False, axis=0)




print(DF)
DF.columns = ['ALL']
DF = DF.reset_index()

DF['METRİKLER'] = INDEX
DF = DF.set_index('METRİKLER')
DF = DF.drop(['index'], axis=1)
DF.to_excel("negatives.xlsx")  

In [ ]:
"""
SHARE METHOD
"""


SHARELİST = []
TOPTOTAL = PRED.iloc[:3 , :] ###share method for first hierarchy 
SHARELİST.append(TOPTOTAL)

#################

Total_hier2FW = PRED.iloc[3:8 , :].sum()
TOTTAL = PRED.iloc[1]
PROP_ROW = TOTTAL / Total_hier2FW
recon_share = PROP_ROW.T * PRED.iloc[3:8 , :] ###share method for second hierarchy 
recon_share = recon_share.fillna(0)
recon_sharehier2FW = recon_share
SHARELİST.append(recon_sharehier2FW)
#################

Total_hier2SS = PRED.iloc[8:13 , :].sum()
TOTTAL = PRED.iloc[2]
PROP_ROW = TOTTAL / Total_hier2SS
recon_share = PROP_ROW.T * PRED.iloc[8:13 , :] ##share method for second hierarchy 
recon_share = recon_share.fillna(0)
recon_sharehier2SS = recon_share

SHARELİST.append(recon_sharehier2SS)

###################

Total_hier3F10 = PRED.iloc[13:16 , :].sum()
TOTTAL = recon_sharehier2FW.iloc[0]
PROP_ROW = TOTTAL / Total_hier3F10
recon_share = PROP_ROW.T * PRED.iloc[13:16 , :] ##share method for third hierarchy 
recon_share = recon_share.fillna(0)
recon_sharehier3F10 = recon_share
SHARELİST.append(recon_sharehier3F10)

###################

Total_hier3F15 = PRED.iloc[16:19 , :].sum()
TOTTAL = recon_sharehier2FW.iloc[1]
PROP_ROW = TOTTAL / Total_hier3F15
recon_share = PROP_ROW.T * PRED.iloc[16:19 , :] ##share method for third hierarchy 
recon_share = recon_share.fillna(0)
recon_sharehier3F15 = recon_share
SHARELİST.append(recon_sharehier3F15)

###################

Total_hier3F20 = PRED.iloc[19:22 , :].sum()
TOTTAL = recon_sharehier2FW.iloc[2]
PROP_ROW = TOTTAL / Total_hier3F20
recon_share = PROP_ROW.T * PRED.iloc[19:22 , :] ##share method for third hierarchy 
recon_share = recon_share.fillna(0)
recon_sharehier3F20 = recon_share
SHARELİST.append(recon_sharehier3F20)

###################

Total_hier3F32 = PRED.iloc[22:25 , :].sum()
TOTTAL = recon_sharehier2FW.iloc[3]
PROP_ROW = TOTTAL / Total_hier3F32
recon_share = PROP_ROW.T * PRED.iloc[22:25 , :] ##share method for third hierarchy 
recon_share = recon_share.fillna(0)
recon_sharehier3F32 = recon_share
SHARELİST.append(recon_sharehier3F32)

###################

Total_hier3F34 = PRED.iloc[25:28 , :].sum()
TOTTAL = recon_sharehier2FW.iloc[4]
PROP_ROW = TOTTAL / Total_hier3F34
recon_share = PROP_ROW.T * PRED.iloc[25:28 , :] ##share method for third hierarchy 
recon_share = recon_share.fillna(0)
recon_sharehier3F34 = recon_share
SHARELİST.append(recon_sharehier3F34)

###################

Total_hier3S10 = PRED.iloc[28:31 , :].sum()
TOTTAL = recon_sharehier2SS.iloc[0]
PROP_ROW = TOTTAL / Total_hier3S10
recon_share = PROP_ROW.T * PRED.iloc[28:31 , :] ##share method for third hierarchy 
recon_share = recon_share.fillna(0)
recon_sharehier3S10 = recon_share
SHARELİST.append(recon_sharehier3S10)

###################

Total_hier3S15 = PRED.iloc[31:34 , :].sum()
TOTTAL = recon_sharehier2SS.iloc[1]
PROP_ROW = TOTTAL / Total_hier3S15
recon_share = PROP_ROW.T * PRED.iloc[31:34 , :] ##share method for third hierarchy 
recon_share = recon_share.fillna(0)
recon_sharehier3S15 = recon_share
SHARELİST.append(recon_sharehier3S15)

###################

Total_hier3S20 = PRED.iloc[34:37 , :].sum()
TOTTAL = recon_sharehier2SS.iloc[2]
PROP_ROW = TOTTAL / Total_hier3S20
recon_share = PROP_ROW.T * PRED.iloc[34:37 , :] ##share method for third hierarchy 
recon_share = recon_share.fillna(0)
recon_sharehier3S20 = recon_share
SHARELİST.append(recon_sharehier3S20)

###################

Total_hier3S32 = PRED.iloc[37:40 , :].sum()
TOTTAL = recon_sharehier2SS.iloc[3]
PROP_ROW = TOTTAL / Total_hier3S32
recon_share = PROP_ROW.T * PRED.iloc[37:40 , :] ##share method for third hierarchy 
recon_share = recon_share.fillna(0)
recon_sharehier3S32 = recon_share
SHARELİST.append(recon_sharehier3S32)

###################

Total_hier3S34 = PRED.iloc[40:43 , :].sum()
TOTTAL = recon_sharehier2SS.iloc[4]
PROP_ROW = TOTTAL / Total_hier3S34
recon_share = PROP_ROW.T * PRED.iloc[40:43 , :] ##share method for third hierarchy 
recon_share = recon_share.fillna(0)
recon_sharehier3S34 = recon_share
SHARELİST.append(recon_sharehier3S34)

SHARE_REC = pd.concat(SHARELİST, axis=0)


Y_base_forecast = PRED.iloc[:, 42:]
Y_real = REAL.iloc[:, 42:]
Y_reconciled = SHARE_REC.iloc[:, 42:]


Y_base_forecast = Y_base_forecast.to_numpy()
Y_real = Y_real.to_numpy()
Y_reconciled = Y_reconciled.to_numpy()



MSE_LST_recon = []
BİAS_LST_recon = []
MAE_LST_recon = []
LOST_MARGİN_PCT_recon = []
INV_COST_PCT_recon = []
TOTAL_COST_PCT_recon = []

MSE_LST_base = []
BİAS_LST_base = []
MAE_LST_base = []
LOST_MARGİN_PCT_base = []
INV_COST_PCT_base = []
TOTAL_COST_PCT_base = []


reconcilations = Y_reconciled.flatten()
expected = Y_real.flatten()
predictions = Y_base_forecast.flatten()

#reconcilations = reconcilations.clip(min=0)

result = np.hstack((expected, reconcilations, predictions))

rechiertot = reconcilations[:9]
expecthiertot = expected[:9]
predhiertot = predictions[:9]

rechier1 = reconcilations[9:27]
expecthier1 = expected[9:27]
predhier1 = predictions[9:27]

rechier2 = reconcilations[27:117]
expecthier2 = expected[27:117]
predhier2 = predictions[27:117]

rechier3 = reconcilations[117:387]
expecthier3 = expected[117:387]
predhier3 = predictions[117:387]

print(sum(rechier1))
print(sum(rechier2))

print(sum(rechier3))

### equality of sum(rechier1), sum(rechier2), sum(rechier3) shows that share method is applied.

In [ ]:
"""

MİNT RECONCİLATİON

"""
### PAPERDAKİ İLE COV ESTİMATE
np.set_printoptions(suppress=True) 
error_X = errordf.iloc[:, :50].to_numpy() # having errors

n = np.ones((30, 1)) # defining 1 matrix
x = error_X - n @ n.T @ error_X / 30 ### calculation in the formula 
var_cov = x @ x.T / 30
var = np.diag(np.diag(var_cov)) ### finding diagonal matirx of var_cov



cov_mat = summing_mat @ var @ summing_mat.T 
inv_cov = np.linalg.pinv(cov_mat)


### MİNT
m, n = summing_mat.shape
m_star = m - n

C = np.ones((13, n))
C_minus = -1 * C
J = np.hstack((np.zeros((n, m_star)), 
               np.eye(n)))
U = np.vstack((np.eye(m_star),
               C_minus.T))

print(C.shape, J.shape, U.shape)

# Calculating P = [J -J * Cov_Mat * U * (U.T * Cov_Mat * U)^(-1) * U.T]
a = J @ cov_mat @ U
b = U.T @ cov_mat @ U
b_inv = np.linalg.inv(b)
c = U.T
beta = J - a @ b_inv @ c
beta

Y_base_forecast = PRED.iloc[:, 42:]
Y_real = REAL.iloc[:, 42:]


Y_reconciled = summing_mat @ beta @ Y_base_forecast.to_numpy()

Y_base_forecast = Y_base_forecast.to_numpy()
Y_real = Y_real.to_numpy()



MSE_LST_recon = []
BİAS_LST_recon = []
MAE_LST_recon = []
LOST_MARGİN_PCT_recon = []
INV_COST_PCT_recon = []
TOTAL_COST_PCT_recon = []

MSE_LST_base = []
BİAS_LST_base = []
MAE_LST_base = []
LOST_MARGİN_PCT_base = []
INV_COST_PCT_base = []
TOTAL_COST_PCT_base = []




reconcilations = Y_reconciled.flatten()
expected = Y_real.flatten()
predictions = Y_base_forecast.flatten()

reconcilations = reconcilations.clip(min=0)

result = np.hstack((expected, reconcilations, predictions))
